In [40]:
import xml.etree.ElementTree as ET
import re
from nltk.corpus import stopwords
import pandas as pd

In [41]:
# read the latest mass production table
# for 6.05 clean
infile_table = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.txt"
infile_brd = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.brd"
outfile_folder = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/FinalBRDs/"

# for 6.10 clean
# infile_brd = "./Output_cleaned_folder/6.10 HTML/6.10 HTML/MassProduction/finalTemplateNew_cleaned.brd"
# infile_table = "./Output_cleaned_folder/6.10 HTML/6.10 HTML/MassProduction/6.10_table_new_cleaned.txt"
# outfile_folder = "./Output_cleaned_folder/6.10 HTML/6.10 HTML/FinalBRDs/"

# for 7.17 clean
# infile_table = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalMassProduction_new_cleaned.txt"
# infile_brd = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new_cleaned.brd"
# outfile_folder = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/FinalBRDs/"

# for 6.05 translate
# infile_table = "./Output_translated_folder/6.05 HTML/6.05 HTML/MassProduction/6.05_translated_no_mark.txt"
# infile_brd = "./Output_cleaned_folder/6.05 HTML/6.05 HTML/MassProduction/6_5_cleaned.brd"
# outfile_folder = "./Output_translated_folder/6.05 HTML/6.05 HTML/FinalBRDs/"

# for 7.17 translate
infile_table = "./Output_translated_folder/7.17 HTML/7.17 HTML/MassProduction/7.17_translated_no_mark.txt"
infile_brd = "./Output_cleaned_folder/7.17 HTML/7.17 HTML/MassProduction/7-17_finalTemplate_new_cleaned.brd"
outfile_folder = "./Output_translated_folder/7.17 HTML/7.17 HTML/FinalBRDs/"

In [42]:
table_new = pd.read_csv(infile_table, sep="\t", index_col=0, keep_default_na=False)

In [43]:
# replace variable with value in the latest mass production table
# iterate column and row in table_new
for column in range(table_new.shape[1]):
    for row in range(table_new.shape[0]):
        content_new = str(table_new.iloc[row, column])
        pattern_variable = "%\((.*?)\)%"
        # count the number of the replacement in one variable(content_new)
        match_variable = [match for match in re.findall(pattern_variable, str(content_new))]
        for i in range(len(match_variable)):
            # find the variable
            variable = "%(" + str(match_variable[i]) + ")%"
            # find the corresponding column name, and then find the value
            column_name = table_new.columns[column]
            try:
                value = table_new.loc[variable, column_name] 
                # print(variable, value)
                content_new = content_new.replace(variable, value)
                table_new.iloc[row, column] = content_new
            except:
                print(column_name, variable + " doesn't exist")

In [44]:
# table_new.head()
for i in range(len(table_new.columns)):
    column_name = table_new.columns[i]
    # print(table_new.columns[i])
    fout = outfile_folder + str(table_new.columns[i]) + ".brd"
    # print(fout)
    count_line = 0
    count_text = 0
    count_name = 0
    with open(infile_brd, 'r') as infile, open(fout, 'w+') as outfile:
        for line in infile:
            line = line.replace('\r', '')
            line_str = str(line)
            # replace massproduce
            # print(line_str)
            if count_name == 0:
                pattern_problem_name_1 = "<ProblemName>(.*?)</ProblemName>"
                match_problem_name_1 = [match for match in re.findall(pattern_problem_name_1, str(line_str))]
                pattern_problem_name_2 = "<ProblemName />"
                match_problem_name_2 = [match for match in re.findall(pattern_problem_name_2, str(line_str))]
                problem_name = "<ProblemName>" + str(column_name) + "</ProblemName>"
                if match_problem_name_1:
                    line_str = line_str.replace(match_problem_name_1[0], str(column_name))
                    # print(line_str)
                    count_name += 1
                elif match_problem_name_2:
                    line_str = line_str.replace(match_problem_name_2[0], problem_name)
                    # print(line_str)
                    count_name += 1
                else:
                    pass
            # replace text in first node
            if count_text == 0:
                pattern_first_node_1 = "<text>(.*?)</text>"
                match_first_node_1 = [match for match in re.findall(pattern_first_node_1, str(line_str))]
                pattern_first_node_2 = "<text />"
                match_first_node_2 = [match for match in re.findall(pattern_first_node_2, str(line_str))]
                node_name = "<text>" + str(column_name) + "</text>"
                if match_first_node_1:
                    line_str = line_str.replace(match_first_node_1[0], str(column_name))
                    # print(line_str)
                    count_text += 1
                elif match_first_node_2:
                    line_str = line_str.replace(match_first_node_2[0], node_name)
                    # print(line_str)
                    count_text += 1
                else:
                    pass
            # count the number of the replacement in one variable(line_str)
            pattern_variable = "%\((.*?)\)%"
            match_variable = [match for match in re.findall(pattern_variable, str(line_str))]
            if match_variable == []:
                line_str = line_str
            else:
                for j in range(len(match_variable)):
                    # find the variable
                    variable = "%(" + str(match_variable[j]) + ")%"
                    # find the corresponding column name, and then find the value
                    try:
                        value = table_new.loc[variable, column_name] 
                        # print(variable, value)
                        line_str = line_str.replace(variable, value).replace("<%", "&lt;%").replace("%>", "%&gt;")
                    except:
                        print(column_name, variable + " doesn't exist") 
            count_line += 1    
            outfile.write(line_str)
        print(fout.split("/")[-1] + " finished")

IntroProblem.brd finished
IntroProblemESP.brd finished
Problem1.brd finished
Problem1ESP.brd finished
Problem2.brd finished
Problem2ESP.brd finished
Problem3.brd finished
Problem3ESP.brd finished
Problem5.brd finished
Problem5ESP.brd finished
Problem6.brd finished
Problem6ESP.brd finished
Problem7.brd finished
Problem7ESP.brd finished
Problem8.brd finished
Problem8ESP.brd finished
Problem9.brd finished
Problem9ESP.brd finished
Problem10.brd finished
Problem10ESP.brd finished
Problem11.brd finished
Problem11ESP.brd finished
Problem12.brd finished
Problem12ESP.brd finished
Problem13.brd finished
Problem13ESP.brd finished
Problem14.brd finished
Problem14ESP.brd finished
Problem15.brd finished
Problem15ESP.brd finished
Problem16.brd finished
Problem16ESP.brd finished
Problem17.brd finished
Problem17ESP.brd finished
Problem18.brd finished
Problem18ESP.brd finished
Problem19.brd finished
Problem19ESP.brd finished
Problem20.brd finished
Problem20ESP.brd finished
Problem21.brd finished
Problem